And now our final model, VGG16 in Gluon

In [1]:
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon
from time import time

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_outputs = 10 # 10 output digits
batch_size = 128 # mini batch
epochs = 64 # total training loops
learning_rate = 0.01 # amount we update parameters

CIFAR images again. Note the need to transform so we are (color channel, x, y).

In [3]:
def transform(data, label):
    data = mx.nd.transpose(data, (2,0,1))
    data = data.astype(np.float32) / 255.0
    return data, label
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.CIFAR10(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.CIFAR10(train=False, transform=transform),
                                     batch_size, shuffle=False)

The parameters for the VGG blocks, 5 blocks in all.

In [4]:
kernels = [3, 3, 3, 3, 3]
filters = [64, 128, 256, 512, 512]
repeats = [2, 2, 3, 3, 3]
pooling = [2, 2, 2, 2, 2]
strides = [2, 2, 2, 2, 2]
dense_units = [4096, 4096]

Again --  you don't have a final softmax *layer*, MxNet handles he softmax inside this loss function, so the output is a straight linear mapping -- no activation function.


In [5]:
vgg16 = gluon.nn.HybridSequential()
with vgg16.name_scope():
    for kernel, filter, pool, stride, repeat in zip(kernels, filters, pooling, strides, repeats):
        for _ in range(0, repeat): #looping and doing convolution and padding repeatedly for each layer
            vgg16.add(gluon.nn.Conv2D(channels=filter, 
                                    kernel_size=kernel,
                                    padding=(kernel//2),
                                    activation='relu'))
        vgg16.add(gluon.nn.MaxPool2D(pool_size=pool, 
                                       strides=stride,
                                       padding=(kernel//2)))

    vgg16.add(gluon.nn.Flatten())

    for units in dense_units:
        vgg16.add(gluon.nn.Dense(units, activation='relu'))
    
    vgg16.add(gluon.nn.Dense(num_outputs))

Gluon requires a context, either `cpu` or `gpu`. You can change this to `cpu` if needed.

In [6]:
ctx = mx.gpu()

Parameter initialization.

In [7]:
vgg16.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

Now let't take a look at the resulting network. We need to feed in a sample batch to infer the network size.

In [8]:
for i, (d, l) in enumerate(train_data):
    print(vgg16.summary(d.as_in_context(ctx)))
    break

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                            (128, 3, 32, 32)               0
        Activation-1   <Symbol hybridsequential0_conv0_relu_fwd>               0
        Activation-2                           (128, 64, 32, 32)               0
            Conv2D-3                           (128, 64, 32, 32)            1792
        Activation-4   <Symbol hybridsequential0_conv1_relu_fwd>               0
        Activation-5                           (128, 64, 32, 32)               0
            Conv2D-6                           (128, 64, 32, 32)           36928
         MaxPool2D-7                           (128, 64, 17, 17)               0
        Activation-8   <Symbol hybridsequential0_conv2_relu_fwd>               0
        Activation-9                          (128, 128, 17, 17)               0
           Conv2D-10        

If you network doesn't change shape, you can `hybridize` it, which makes Gluon run in a precomplied mode much like Keras.

In [9]:
vgg16.hybridize()

And as always, learning is done with an optimizer and a loss function, learning a classifier with categorical cross entropy.

In [10]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(vgg16.collect_params(), 'sgd', {'learning_rate': learning_rate})

Accuracy!

In [11]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

And the training loop. 

In [12]:
smoothing_constant = .01
moving_loss = 0.0

for e in range(epochs):
    start = time()
    for i, (d, l) in enumerate(train_data):
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        with autograd.record():
            output = vgg16(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        #  Keep a moving average of the losses
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)
    elapsed = time() - start

    test_accuracy = evaluate_accuracy(test_data, vgg16)
    train_accuracy = evaluate_accuracy(train_data, vgg16)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s, Time %f" % (e, moving_loss, train_accuracy, test_accuracy, elapsed))

Epoch 0. Loss: 2.3023834320806125, Train_acc 0.11012, Test_acc 0.1092, Time 25.231127
Epoch 1. Loss: 2.302035459340422, Train_acc 0.12996, Test_acc 0.1306, Time 24.935430
Epoch 2. Loss: 2.3014975379081557, Train_acc 0.18354, Test_acc 0.1889, Time 24.405735
Epoch 3. Loss: 2.3002735904957174, Train_acc 0.18038, Test_acc 0.1811, Time 22.922612
Epoch 4. Loss: 2.2966594224527874, Train_acc 0.2035, Test_acc 0.2073, Time 22.926140
Epoch 5. Loss: 2.2814693055800324, Train_acc 0.19408, Test_acc 0.1907, Time 23.609636
Epoch 6. Loss: 2.1740437756168203, Train_acc 0.19998, Test_acc 0.2026, Time 22.846272
Epoch 7. Loss: 2.063757028394381, Train_acc 0.23552, Test_acc 0.2366, Time 25.318111
Epoch 8. Loss: 2.01008239545407, Train_acc 0.26302, Test_acc 0.2725, Time 22.835264
Epoch 9. Loss: 1.9519827310537334, Train_acc 0.29422, Test_acc 0.2997, Time 23.640966
Epoch 10. Loss: 1.9220144043531193, Train_acc 0.28886, Test_acc 0.2941, Time 24.765907
Epoch 11. Loss: 1.8752749971809894, Train_acc 0.3061, Test

Training this one a little longer, syou can see the training accuracy shoots way up -- but there is a big gap between the training and testing accuracy. This is what we call overfitting -- the model has in a sense memorized the inputs, making a very expensive neural network hashtable!

In the next section we'll investigate techniques to overcome this.